In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

image = cv2.imread('/mnt/f/valorant_VODs/gray_roi_4.png', cv2.IMREAD_GRAYSCALE)


kernel = np.ones((5,5),np.uint8)


thresholds = list(range(80, 110))


plt.figure(figsize=(5, 5))
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.show()


n = len(thresholds)
cols = 5
rows = n // cols + 1

fig, ax = plt.subplots(rows, cols, figsize=(20, rows*4))

for i, threshold in enumerate(thresholds):
    binary_thresh = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)[1]
    dilated = cv2.dilate(binary_thresh, kernel, iterations = 1)

    mask = binary_thresh > 0
    image_copy = image.copy()
    image_copy[~mask] = 0

    row = i // cols
    col = i % cols
    ax[row, col].imshow(image_copy, cmap='gray')
    ax[row, col].set_title('Threshold = ' + str(threshold))

for j in range(i+1, rows*cols):
    fig.delaxes(ax.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import pytesseract
import numpy as np
from matplotlib import pyplot as plt
from collections import deque, Counter

ocr_roi = (978, 7, 1000, 24)
crop_roi = (30, 50, 410, 410)


cap = cv2.VideoCapture("/mnt/f/valorant_VODs/test2.mp4")

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

fps = 3
width = crop_roi[2] - crop_roi[0]
height = crop_roi[3] - crop_roi[1]

video_writers = {}
current_number = 1
frame_counter = 0

def preprocess_image(image):
    threshold = 103
    binary_thresh = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)[1]
    mask = binary_thresh > 0
    image_copy = image.copy()
    image_copy[~mask] = 0
    return image_copy

def numbercheck(number):
    number = number.strip()
    if number.isdigit():
        num = int(number)
        return 0 <= num - int(current_number) <= 1
    return False


buffer_size = 10
buffer = deque(maxlen=buffer_size)

while(cap.isOpened()):
    ret, frame = cap.read()
    frame_counter += 1
    if ret == True:
        if frame_counter % 10 == 0:
            
            roi = frame[ocr_roi[1]:ocr_roi[3], ocr_roi[0]:ocr_roi[2]]
            gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray_roi_processed = preprocess_image(gray_roi)
            number = pytesseract.image_to_string(gray_roi_processed, config='--psm 6 -c tessedit_char_whitelist=0123456789')
            
            # print(number,end=' ')
            # if frame_counter % 50 == 0:
            #     print(frame_counter)

            if numbercheck(number):
                buffer.append(number.strip())
            
            
            if buffer:
                most_common_number, count = Counter(buffer).most_common(1)[0]
                
                if count/buffer_size >= 0.5:
                    current_number = most_common_number
                
                # writing the frame to the video
                if current_number not in video_writers:
                    # print('number:'+current_number)
                    video_writers[current_number] = cv2.VideoWriter('/mnt/f/valorant_VODs/test2_round' + str(current_number).strip() + '.mp4', fourcc, fps, (width, height))

                cropped_frame = frame[crop_roi[1]:crop_roi[3], crop_roi[0]:crop_roi[2]]
                video_writers[current_number].write(cropped_frame)
    else:
        break

cap.release()
for writer in video_writers.values():
    writer.release()

cv2.destroyAllWindows()

In [9]:
import cv2
import os

video_file_path = "/mnt/f/valorant_VODs/VCT_Masters_Tokyo/T1 vs. EDG - VALORANT Masters Tokyo - Group A - Map 2.mp4"
output_folder_path = "/mnt/f/valorant_VODs/ocr_dataset"

cap = cv2.VideoCapture(video_file_path)
frame_count = 0
thresh_value = 103

if not cap.isOpened():
    print("Video file could not be opened")
    exit(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # if frame_count >= 30*60*10:
    #     break

    if frame_count % 10 == 0:
        cropped_frame = frame[10:25, 979:999]
        gray_frame = cv2.cvtColor(cropped_frame,cv2.COLOR_BGR2GRAY)
        binary_frame = cv2.threshold(gray_frame, thresh_value, 255, cv2.THRESH_BINARY)[1]
        output_path = os.path.join(output_folder_path, f'T1vsEDG_GA_M2_frame_{int(frame_count/10)}.jpg')
        cv2.imwrite(output_path, binary_frame)

    frame_count += 1

cap.release()


In [7]:
import cv2
import os
import glob

input_dir = "/mnt/f/valorant_VODs/ocr_dataset"

thresh_value = 103

image_files = glob.glob(os.path.join(input_dir, "*.jpg"))

for image_file in image_files:
    
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

    img_binary = cv2.threshold(img, thresh_value, 255, cv2.THRESH_BINARY)[1]

    cv2.imwrite(image_file, img_binary)


In [4]:
import os

for i in range(1,34):
    dir_path = os.path.join("/mnt/f/valorant_VODs/ocr_dataset",str(i))
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [14]:
from collections import defaultdict, deque

def topological_sort(graph, N):
    indegree = {node: 0 for node in range(N)}
    for node in graph:
        for neighbour in graph[node]:
            indegree[neighbour] += 1

    queue = deque([node for node in indegree if indegree[node] == 0])
    topological_order = []

    while queue:
        node = queue.popleft()
        topological_order.append(node)

        for neighbour in graph[node]:
            indegree[neighbour] -= 1

            if indegree[neighbour] == 0:
                queue.append(neighbour)

    return topological_order

def create_ranking(relative):
    N = len(relative)
    graph = defaultdict(list)

    for i in range(N):
        for j in range(N):
            if relative[i][j] == 0:
                graph[j].append(i)
            elif relative[i][j] == 1:
                graph[i].append(j)

    ranking = topological_sort(graph, N)

    return ranking

relative = [
    [2, 0, 1, 0, 2, 1, 0, 1, 0, 1],
    [1, 2, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 2, 0, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 2, 1, 2, 0, 1, 0, 1],
    [2, 1, 1, 0, 2, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 2, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 2, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 2, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 2]
]


ranking = create_ranking(relative)
print(ranking)

[]


In [20]:
import numpy as np

def create_ranking(relative):
    scores = np.sum(np.array(relative) == 0, axis=1) - np.sum(np.array(relative) == 1, axis=1)
    ranking = np.argsort(scores)[::-1]
    return ranking.tolist()

relative = [
    [2, 0, 1, 0, 2, 1, 0, 1, 0, 1],
    [1, 2, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 2, 0, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 2, 1, 2, 0, 1, 0, 1],
    [2, 1, 1, 0, 2, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 2, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 2, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 2, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 2]
]

"""
A<B
A>C
A?D
B>C
B>D
C<D
"""

relative_importance = [
    [2, 1, 0, 2],  # Aの行: A<B, A>C, A?D
    [0, 2, 0, 0],  # Bの行: B>A, B>C, B>D
    [1, 1, 2, 1],  # Cの行: C<A, C<B, C<D
    [2, 2, 0, 2],  # Dの行: D?A, D<B, D>C
]

relative3 = [
    [2,1,0,0,1,0,0,0,1,2],
[0,2,0,0,2,0,0,0,2,0],
[1,1,2,2,1,2,2,2,1,1],
[1,1,2,2,1,2,2,2,1,1],
[0,2,0,0,2,0,0,0,2,0],
[1,1,2,2,1,2,2,2,1,1],
[1,1,2,2,1,2,2,2,1,1],
[1,1,2,2,1,2,2,2,1,1],
[0,2,0,0,2,0,0,0,2,0],
[2,1,0,0,1,0,0,0,1,2],
]


ranking = create_ranking(relative)
print(ranking)
ranking = create_ranking(relative2)
print(ranking)
ranking = create_ranking(relative3)
print(ranking)


[9, 7, 5, 2, 0, 1, 4, 3, 8, 6]
[1, 3, 0, 2]
[8, 4, 1, 9, 0, 7, 6, 5, 3, 2]


In [17]:
import numpy as np

def create_ranking(relative):
    # 優劣のスコアを計算
    scores = np.sum(np.array(relative) == 0, axis=1) - np.sum(np.array(relative) == 1, axis=1)
    
    # 基本的なランキングを作成
    ranking = np.argsort(scores)[::-1]
    
    # "判断できない"評価を考慮に入れてランキングを更新
    for i in range(len(relative)):
        for j in range(i+1, len(relative)):
            if relative[i][j] == 2 or relative[j][i] == 2:
                if ranking[i] != ranking[j]:
                    lower_rank = min(ranking[i], ranking[j])
                    ranking[i] = ranking[j] = lower_rank
    
    return ranking.tolist()

relative = [
    [2, 1, 0, 2],  # Aの行: A<B, A>C, A?D
    [0, 2, 0, 0],  # Bの行: B>A, B>C, B>D
    [1, 1, 2, 1],  # Cの行: C<A, C<B, C<D
    [2, 2, 0, 2],  # Dの行: D?A, D<B, D>C
]

ranking = create_ranking(relative)
print(ranking)

[1, 1, 0, 1]
